In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [2]:
WINDOW_IN_MIN = 180   # 3 hours
WINDOW_OUT_MIN = 60   # 1 hour ahead

flare_order = {"A": 0, "B": 1, "C": 2, "M": 3, "X": 4}

def get_highest_class(classes):
    if len(classes) == 0:
        return "A"
    return max(classes, key=lambda c: flare_order[c])

def make_future_labels(index, flare_df, horizon_min=60):
    """
    index: DatetimeIndex of the 1-min irradiance series
    flare_df: dataframe with ['flare_time', 'class_letter']
    """
    fl_times = flare_df['flare_time'].values
    fl_classes = flare_df['class_letter'].values
    n_events = len(fl_times)

    labels = []
    j = 0

    for t in index.values:
        # advance pointer past any flares before t
        while j < n_events and fl_times[j] < t:
            j += 1

        h_end = t + np.timedelta64(horizon_min, 'm')
        k = j
        classes = []

        while k < n_events and fl_times[k] <= h_end:
            classes.append(fl_classes[k])
            k += 1

        labels.append(get_highest_class(classes))

    return np.array(labels)

def build_windows(irrad_1m, labels, window_in_min=180):
    data = irrad_1m[['short_xray', 'long_xray']].values
    X_list, y_list = [], []

    for i in range(window_in_min, len(irrad_1m)):
        x_window = data[i-window_in_min:i, :]
        X_list.append(x_window)
        y_list.append(labels[i])

    X = np.stack(X_list)
    y = np.array(y_list)
    return X, y


In [3]:
def process_year(year, base_irrad_dir="", base_flsum_dir=""):
    print(f"\n=== Processing year {year} ===")
    irrad_path = f"{base_irrad_dir}irrad_{year}.nc"
    flsum_path = f"{base_flsum_dir}flsum_{year}.nc"

    # 1) Load irrad
    irrad_ds = xr.open_dataset(irrad_path)
    irrad_df = irrad_ds[['a_flux', 'b_flux']].to_dataframe().reset_index()

    irrad_df.rename(columns={
        'time': 'timestamp',
        'a_flux': 'short_xray',
        'b_flux': 'long_xray'
    }, inplace=True)

    irrad_df['timestamp'] = pd.to_datetime(irrad_df['timestamp'])
    irrad_df.set_index('timestamp', inplace=True)

    # 2) Resample to 1-min
    irrad_1m = irrad_df.resample('1min').mean().dropna()
    print("1-min irradiance shape:", irrad_1m.shape)

    # 3) Load flare summary
    flsum_ds = xr.open_dataset(flsum_path)
    flsum_df = flsum_ds[['flare_class']].to_dataframe().reset_index()

    flsum_df.rename(columns={'time': 'flare_time'}, inplace=True)
    flsum_df['flare_time'] = pd.to_datetime(flsum_df['flare_time'])
    flsum_df['class_letter'] = flsum_df['flare_class'].astype(str).str[0]

    valid_classes = ['A', 'B', 'C', 'M', 'X']
    flsum_df = flsum_df[flsum_df['class_letter'].isin(valid_classes)].copy()
    flsum_df.sort_values('flare_time', inplace=True)
    flsum_df.reset_index(drop=True, inplace=True)

    print("Number of flare events:", len(flsum_df))

    # 4) Future labels (next 1 hour)
    labels = make_future_labels(irrad_1m.index, flsum_df, horizon_min=WINDOW_OUT_MIN)
    print("Labels shape:", labels.shape)

    # 5) Build sliding windows
    X_year, y_year = build_windows(irrad_1m, labels, window_in_min=WINDOW_IN_MIN)
    print("Year", year, "X shape:", X_year.shape, "y shape:", y_year.shape)

    return X_year, y_year

In [4]:
base_irrad_dir = "C:/Users/ruthw/Desktop/irrad/"
base_flsum_dir = "C:/Users/ruthw/Desktop/flsum/"

In [5]:
YEARS = [2012, 2013, 2014]  # adjust to what you have

X_list = []
y_list = []

for year in YEARS:
    X_y, y_y = process_year(year, base_irrad_dir, base_flsum_dir)
    X_list.append(X_y)
    y_list.append(y_y)

X_all = np.concatenate(X_list, axis=0)
y_all = np.concatenate(y_list, axis=0)

print("Combined X:", X_all.shape)
print("Combined y:", y_all.shape)
print(pd.Series(y_all).value_counts())


=== Processing year 2012 ===
1-min irradiance shape: (501498, 2)
Number of flare events: 3141
Labels shape: (501498,)
Year 2012 X shape: (501318, 180, 2) y shape: (501318,)

=== Processing year 2013 ===
1-min irradiance shape: (522024, 2)
Number of flare events: 2975
Labels shape: (522024,)
Year 2013 X shape: (521844, 180, 2) y shape: (521844,)

=== Processing year 2014 ===
1-min irradiance shape: (512719, 2)
Number of flare events: 3226
Labels shape: (512719,)
Year 2014 X shape: (512539, 180, 2) y shape: (512539,)
Combined X: (1535701, 180, 2)
Combined y: (1535701,)
A    1072419
C     326234
B      95096
M      38960
X       2992
Name: count, dtype: int64


In [6]:
# Clean
X_all[~np.isfinite(X_all)] = 0.0
X_all = np.clip(X_all, 0.0, None)

# Log-transform
X_all = np.log10(X_all + 1e-10)

# Optional: global standardization
mean = X_all.mean(axis=(0, 1), keepdims=True)
std  = X_all.std(axis=(0, 1), keepdims=True) + 1e-6
X_all = (X_all - mean) / std

print("After transform, any NaN?", np.isnan(X_all).any(), " any Inf?", np.isinf(X_all).any())

After transform, any NaN? False  any Inf? False


In [7]:
np.save("X_multi.npy", X_all)
np.save("y_multi.npy", y_all)

In [8]:
X = np.load("X_multi.npy")
y = np.load("y_multi.npy")

# Manual encoding to keep class order fixed
label_map = {'A': 0, 'B': 1, 'C': 2, 'M': 3, 'X': 4}
y_encoded = np.array([label_map[c] for c in y])
inv_label_map = {v: k for k, v in label_map.items()}

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_encoded, test_size=0.30, random_state=42, stratify=y_encoded
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

classes = np.array([0,1,2,3,4])
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)
class_weights = torch.tensor(class_weights, dtype=torch.float32)
print("Class weights:", class_weights)

Class weights: tensor([  0.2864,   3.2298,   0.9415,   7.8835, 102.6734])


# MODELS

In [9]:
print("X:", X.shape)
print("y:", y.shape)
pd.Series(y).value_counts()

X: (1535701, 180, 2)
y: (1535701,)


A    1072419
C     326234
B      95096
M      38960
X       2992
Name: count, dtype: int64

In [10]:
label_map = {'A': 0, 'B': 1, 'C': 2, 'M': 3, 'X': 4}
inv_label_map = {v: k for k, v in label_map.items()}

y_encoded = np.array([label_map[c] for c in y])

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_encoded, test_size=0.30, random_state=42, stratify=y_encoded
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

Train: (1074990, 180, 2) Val: (230355, 180, 2) Test: (230356, 180, 2)


In [12]:
classes = np.array([0,1,2,3,4])

raw_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)

# Tame extreme imbalance
scaled_weights = np.sqrt(raw_weights)
scaled_weights = scaled_weights / scaled_weights.min()

class_weights = torch.tensor(scaled_weights, dtype=torch.float32)

print("Raw:", raw_weights)
print("Scaled:", scaled_weights)

Raw: [  0.28639937   3.22979855   0.94147063   7.88347023 102.67335244]
Scaled: [ 1.          3.35816237  1.81308165  5.24653638 18.93402056]


In [13]:
class GOESDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.int64)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # (180, 2) → (2, 180)
        x = self.X[idx].transpose(1, 0)
        return x, self.y[idx]

train_loader = DataLoader(GOESDataset(X_train, y_train), batch_size=256, shuffle=True)
val_loader   = DataLoader(GOESDataset(X_val, y_val), batch_size=512, shuffle=False)
test_loader  = DataLoader(GOESDataset(X_test, y_test), batch_size=512, shuffle=False)

In [23]:
# CNN

In [14]:
class CNN1D(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.conv1 = nn.Conv1d(2, 32, kernel_size=5, padding=2)
        self.bn1   = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, padding=2)
        self.bn2   = nn.BatchNorm1d(64)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, padding=2)
        self.bn3   = nn.BatchNorm1d(128)

        self.pool = nn.MaxPool1d(2)
        self.dropout = nn.Dropout(0.3)

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(128, n_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = self.global_pool(x).squeeze(-1)
        x = self.dropout(x)
        return self.fc(x)

In [15]:
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=1.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, logits, targets):
        ce = F.cross_entropy(logits, targets, weight=self.alpha, reduction='none')
        pt = torch.exp(-ce)
        loss = ((1 - pt) ** self.gamma) * ce
        return loss.mean()

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN1D(n_classes=5).to(device)
criterion = FocalLoss(alpha=class_weights.to(device), gamma=1.0)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def run_epoch(loader, train=True):
    if train:
        model.train()
    else:
        model.eval()

    total_loss, correct, total = 0, 0, 0

    for Xb, yb in loader:
        Xb, yb = Xb.to(device), yb.to(device)
        if train:
            optimizer.zero_grad()

        with torch.set_grad_enabled(train):
            out = model(Xb)
            loss = criterion(out, yb)
            if train:
                loss.backward()
                optimizer.step()

        total_loss += loss.item() * len(yb)
        preds = out.argmax(1)
        correct += (preds == yb).sum().item()
        total += len(yb)

    return total_loss / total, correct / total

In [17]:
EPOCHS = 10
for ep in range(1, EPOCHS+1):
    train_loss, train_acc = run_epoch(train_loader, train=True)
    val_loss, val_acc     = run_epoch(val_loader, train=False)

    print(f"Epoch {ep:02d} | "
          f"Train loss {train_loss:.4f}, acc {train_acc:.3f} | "
          f"Val loss {val_loss:.4f}, acc {val_acc:.3f}")

Epoch 01 | Train loss 1.1748, acc 0.609 | Val loss 1.2101, acc 0.512
Epoch 02 | Train loss 1.1265, acc 0.625 | Val loss 1.1417, acc 0.648
Epoch 03 | Train loss 1.1003, acc 0.631 | Val loss 1.0935, acc 0.703
Epoch 04 | Train loss 1.0744, acc 0.633 | Val loss 1.2434, acc 0.627
Epoch 05 | Train loss 1.0480, acc 0.633 | Val loss 1.0287, acc 0.658
Epoch 06 | Train loss 1.0214, acc 0.634 | Val loss 1.0754, acc 0.690
Epoch 07 | Train loss 0.9962, acc 0.636 | Val loss 1.8268, acc 0.382
Epoch 08 | Train loss 0.9717, acc 0.638 | Val loss 1.2128, acc 0.526
Epoch 09 | Train loss 0.9466, acc 0.642 | Val loss 1.0219, acc 0.573
Epoch 10 | Train loss 0.9247, acc 0.644 | Val loss 1.5312, acc 0.470


In [18]:
from sklearn.metrics import classification_report, confusion_matrix

model.eval()
all_preds, all_true = [], []

with torch.no_grad():
    for Xb, yb in test_loader:
        Xb, yb = Xb.to(device), yb.to(device)
        out = model(Xb)
        preds = out.argmax(1)
        all_preds.append(preds.cpu().numpy())
        all_true.append(yb.cpu().numpy())

all_preds = np.concatenate(all_preds)
all_true  = np.concatenate(all_true)

print(classification_report(all_true, all_preds,
                            target_names=['A','B','C','M','X']))

print(confusion_matrix(all_true, all_preds))

              precision    recall  f1-score   support

           A       0.78      0.53      0.63    160863
           B       0.10      0.59      0.17     14265
           C       0.44      0.24      0.31     48935
           M       0.27      0.36      0.31      5844
           X       0.32      0.21      0.25       449

    accuracy                           0.47    230356
   macro avg       0.38      0.39      0.33    230356
weighted avg       0.65      0.47      0.53    230356

[[85686 58685 13527  2861   104]
 [ 5847  8396    15     7     0]
 [16836 17529 11798  2736    36]
 [ 1524   787  1397  2079    57]
 [  113    23   120    99    94]]


In [19]:
from sklearn.metrics import classification_report, confusion_matrix

# Ground truth: ≥C
y_true_C = (all_true >= 2).astype(int)
y_pred_C = (all_preds >= 2).astype(int)

print("Binary ≥C classification report:")
print(classification_report(
    y_true_C,
    y_pred_C,
    target_names=["<C (A/B)", "≥C (C/M/X)"]
))

print("Confusion matrix (≥C):")
print(confusion_matrix(y_true_C, y_pred_C))

Binary ≥C classification report:
              precision    recall  f1-score   support

    <C (A/B)       0.81      0.91      0.86    175128
  ≥C (C/M/X)       0.53      0.33      0.41     55228

    accuracy                           0.77    230356
   macro avg       0.67      0.62      0.63    230356
weighted avg       0.74      0.77      0.75    230356

Confusion matrix (≥C):
[[158614  16514]
 [ 36812  18416]]


In [20]:
# Ground truth: ≥M
y_true_M = (all_true >= 3).astype(int)
y_pred_M = (all_preds >= 3).astype(int)

print("Binary ≥M classification report:")
print(classification_report(
    y_true_M,
    y_pred_M,
    target_names=["<M (A/B/C)", "≥M (M/X)"]
))

print("Confusion matrix (≥M):")
print(confusion_matrix(y_true_M, y_pred_M))

Binary ≥M classification report:
              precision    recall  f1-score   support

  <M (A/B/C)       0.98      0.97      0.98    224063
    ≥M (M/X)       0.29      0.37      0.32      6293

    accuracy                           0.96    230356
   macro avg       0.64      0.67      0.65    230356
weighted avg       0.96      0.96      0.96    230356

Confusion matrix (≥M):
[[218319   5744]
 [  3964   2329]]
